Major Project -> April-may 202

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import xgboost as xgb

In [2]:
data = pd.read_excel('Data_Train.xlsx')
#print(data.isnull().sum())
data = data.dropna(axis=0, how = 'any')

In [3]:
#Execute only once.
dataVals = data['Power']
vals = []
nulenc = 0
nulindex = []
rowindex = -1
for val in dataVals:
    rowindex+= 1
    ind = 0
    for index in range(0,len(val)):
        if(val[index] == ' ' or val[index] == 'C' or val[index] == 'k' or val[index] == 'b' ):
            ind = index
            break
    val = val[:index]
    if(val=='null' or val == '0' or val == '0.0'):
        nulenc += 1
        val = 0
        nulindex.append(rowindex)
    
    vals.append(val)
y3 = np.array(vals)
y3 = y3.astype(np.float)
print(nulenc)

103


In [4]:
#DO NOT EXECUTE MORE THAN ONCE
dataValues = data['Mileage']
vals = []
rowindex = -1
for val in dataValues:
    rowindex += 1
    ind = 0
    for index in range(0,len(val)):
        if(val[index] == ' ' or val[index] == 'C' or val[index] == 'k' or val[index] == 'b' ):
            ind = index
            break
    val = val[:index]
    vals.append(val)
    if(val == '0' or val == '0.0'):
        nulindex.append(rowindex)
y = np.array(vals)
y = y.astype(np.float)


In [5]:
#Execute only once.
dataVals = data['Engine']
vals = []
rowindex = -1
for val in dataVals:
    rowindex+=1
    ind = 0
    for index in range(0,len(val)):
        if(val[index] == ' ' or val[index] == 'C' or val[index] == 'k' or val[index] == 'b' ):
            ind = index
            break
    val = val[:index]
    vals.append(val)
    if(val == '0' or val == '0.0'):
        print(rowindex)
        nulindex.append(rowindex)
y2 = np.array(vals)
y2 = y2.astype(np.float)


In [6]:
finalData = pd.DataFrame()
loc = data['Location']
loc_enc = pd.get_dummies(data=loc)
finalData = finalData.append(loc_enc)

year= data['Year']
year_enc = pd.get_dummies(data=year)
finalData = finalData.join(year_enc)

finalData = finalData.join(data['Kilometers_Driven'])

fueltype = data['Fuel_Type']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

fueltype = data['Transmission']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

fueltype = data['Owner_Type']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

fueltype = data['Seats']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

data['Mileage'] = y
finalData = finalData.join(data['Mileage'])

data['Engine'] = y2
finalData = finalData.join(data['Engine'])

data['Power'] = y3
finalData = finalData.join(data['Power'])

#finalData.drop(nulindex)
#print(nulindex)

finalData = finalData.drop(nulindex)
dataFin = data.drop(nulindex)




X,y = finalData.iloc[: , :], dataFin.iloc[:, -1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#lrset = [0.1, 0.01, 0.5]
#for lr in lrset:  #manually using for loops instead of gridsearchCv
lr = 0.5      
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = lr,
                          max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print()
print("RMSE: %f" % (rmse))
print()


params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': lr,
          'max_depth': 10, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=10,
                    num_boost_round=50,early_stopping_rounds=1000,metrics="rmse", as_pandas=True, seed=123)
print()
print()
print((cv_results["test-rmse-mean"]).tail(1))

print()
print()
print()
print()

[19:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.

RMSE: 4.520866

[19:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/reg

In [7]:
#For accuracy reasons, the same segment of the code is run twice.
finalData = pd.DataFrame()
loc = data['Location']
loc_enc = pd.get_dummies(data=loc)
finalData = finalData.append(loc_enc)

year= data['Year']
year_enc = pd.get_dummies(data=year)
finalData = finalData.join(year_enc)

finalData = finalData.join(data['Kilometers_Driven'])

fueltype = data['Fuel_Type']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

fueltype = data['Transmission']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

fueltype = data['Owner_Type']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

fueltype = data['Seats']
ft_enc = pd.get_dummies(data = fueltype)
finalData = finalData.join(ft_enc)

data['Mileage'] = y
finalData = finalData.join(data['Mileage'])

data['Engine'] = y2
finalData = finalData.join(data['Engine'])

data['Power'] = y3
finalData = finalData.join(data['Power'])

#finalData.drop(nulindex)
#print(nulindex)

finalData = finalData.drop(nulindex)
dataFin = data.drop(nulindex)
print(finalData.columns)



X,y = finalData.iloc[: , :], dataFin.iloc[:, -1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#lrset = [0.1, 0.01, 0.5]
#for lr in lrset:  #manually using for loops instead of gridsearchCv
lr = 0.5      
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = lr,
                          max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print()
print("RMSE: %f" % (rmse))
print()


params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': lr,
          'max_depth': 10, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=10,
                    num_boost_round=50,early_stopping_rounds=1000,metrics="rmse", as_pandas=True, seed=123)
print()
print()
print((cv_results["test-rmse-mean"]).tail(1))

print()
print()
print()
print()

Index([        'Ahmedabad',         'Bangalore',           'Chennai',
              'Coimbatore',             'Delhi',         'Hyderabad',
                  'Jaipur',             'Kochi',           'Kolkata',
                  'Mumbai',              'Pune',                1998,
                      1999,                2000,                2001,
                      2002,                2003,                2004,
                      2005,                2006,                2007,
                      2008,                2009,                2010,
                      2011,                2012,                2013,
                      2014,                2015,                2016,
                      2017,                2018,                2019,
       'Kilometers_Driven',               'CNG',            'Diesel',
                     'LPG',            'Petrol',         'Automatic',
                  'Manual',             'First',    'Fourth & Above',
                  'S

In [11]:
finTest = pd.read_excel('Data_Train.xlsx')
finTest.dropna(how = 'any')
#Execute only once.
dataValsF = finTest['Power']
vals = []
nulenc = 0
nulindex = []
rowindex = -1
for val in dataValsF:
    #print("val = " + val)
    rowindex+= 1
    ind = 0
    val = str(val)
    for index in range(0,len(val)):
        if(val[index] == ' ' or val[index] == 'C' or val[index] == 'k' or val[index] == 'b' ):
            ind = index
            break
    val = val[:index]
    if(val=='null' or val == '0' or val == '0.0' or val =='na'):
        nulenc += 1
        val = 0
        nulindex.append(rowindex)
    
    vals.append(val)
y3F = np.array(vals)
y3F = y3F.astype(np.float)

#DO NOT EXECUTE MORE THAN ONCE
dataValuesF = finTest['Mileage']
vals = []
rowindex = -1
for val in dataValuesF:
    rowindex += 1
    ind = 0
    val = str(val)
    for index in range(0,len(val)):
        if(val[index] == ' ' or val[index] == 'C' or val[index] == 'k' or val[index] == 'b' ):
            ind = index
            break
    val = val[:index]
    if(val == '0' or val == '0.0' or val == 'na'):
        val = 0
        nulindex.append(rowindex)
    vals.append(val)
yF = np.array(vals)
yF = yF.astype(np.float)

#Execute only once.
dataValsF = finTest['Engine']
vals = []
rowindex = -1
for val in dataValsF:
    rowindex+=1
    ind = 0
    val = str(val)
    for index in range(0,len(val)):
        if(val[index] == ' ' or val[index] == 'C' or val[index] == 'k' or val[index] == 'b' ):
            ind = index
            break
    val = val[:index]
    
    if(val == '0' or val == '0.0' or val == 'na'):
        val = 0
        nulindex.append(rowindex)
    vals.append(val)
y2F = np.array(vals)
y2F = y2F.astype(np.float)

finalTestData = pd.DataFrame()
loc = finTest['Location']
loc_enc = pd.get_dummies(data=loc)
finalTestData = finalTestData.append(loc_enc)

year= finTest['Year']
year_enc = pd.get_dummies(data=year)
finalTestData = finalTestData.join(year_enc)

finalTestData = finalTestData.join(finTest['Kilometers_Driven'])

fueltype = finTest['Fuel_Type']
ft_enc = pd.get_dummies(data = fueltype)
finalTestData = finalTestData.join(ft_enc)

fueltype = finTest['Transmission']
ft_enc = pd.get_dummies(data = fueltype)
finalTestData = finalTestData.join(ft_enc)

fueltype = finTest['Owner_Type']
ft_enc = pd.get_dummies(data = fueltype)
finalTestData = finalTestData.join(ft_enc)

fueltype = finTest['Seats']
ft_enc = pd.get_dummies(data = fueltype)
finalTestData = finalTestData.join(ft_enc)

finTest['Mileage'] = yF
finalTestData = finalTestData.join(finTest['Mileage'])

finTest['Engine'] = y2F
finalTestData = finalTestData.join(finTest['Engine'])

finTest['Power'] = y3F
finalTestData = finalTestData.join(finTest['Power'])

#finalData.drop(nulindex)
#print(nulindex)

finalTestData = finalTestData.drop(nulindex)

finalTestData = finalTestData.drop(['Electric'], axis = 1)
finalTestData

preds = xg_reg.predict(finalTestData)

preds[[11]]

array([7.603362], dtype=float32)